In [3]:
import pathlib

import numpy as np
from rich.pretty import pprint

from oqd_dataschema.base import Dataset
from oqd_dataschema.datastore import Datastore
from oqd_dataschema.groups import (
    ExpectationValueDataGroup,
    MeasurementOutcomesDataGroup,
    SinaraRawDataGroup,
)

In [5]:
raw = SinaraRawDataGroup(
    camera_images=Dataset(shape=(3, 2, 2), dtype="float32"),
    attrs={"date": "2025-03-26", "version": 0.1}
)
pprint(raw)

SinaraRawDataGroup(
│   attrs={'date': '2025-03-26', 'version': 0.1},
│   camera_images=Dataset(dtype='float32', shape=(3, 2, 2), data=None, attrs={})
)

In [7]:
raw.camera_images.data = np.random.uniform(size=(3, 2, 2)).astype("float32")
pprint(raw)

SinaraRawDataGroup(
│   attrs={'date': '2025-03-26', 'version': 0.1},
│   camera_images=Dataset(
│   │   dtype='float32',
│   │   shape=(3, 2, 2),
│   │   data=array([[[0.31180453, 0.20829408],
│   │   [0.05447883, 0.5814612 ]],
│      
│      [[0.39647558, 0.5823655 ],
│   │   [0.8776862 , 0.23722762]],
│      
│      [[0.2397174 , 0.511561  ],
│   │   [0.45516378, 0.07079336]]], dtype=float32),
│   │   attrs={}
│   )
)

In [10]:
# raw.camera_images.data = np.random.uniform(size=(3, 4, 4)).astype("float32")
# raw.camera_images.data = np.random.uniform(size=(3, 2, 2)).astype("float64")
raw.camera_images.data = np.random.uniform(size=(3, 2, 2)).astype("float32")

In [11]:
data = Datastore(groups={"raw": raw})
pprint(data)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(3, 2, 2),
│   │   │   │   data=array([[[0.24662715, 0.86970806],
│   │   [0.34476417, 0.90156907]],
│      
│      [[0.17104147, 0.60216415],
│   │   [0.50629896, 0.80135703]],
│      
│      [[0.9071514 , 0.8023807 ],
│   │   [0.68783087, 0.0879358 ]]], dtype=float32),
│   │   │   │   attrs={}
│   │   │   )
│   │   )
│   }
)

In [12]:
def process_raw(raw: SinaraRawDataGroup) -> MeasurementOutcomesDataGroup:
    processed = MeasurementOutcomesDataGroup(
        outcomes=Dataset(
            data=np.round(raw.camera_images.data.mean(axis=(1, 2))),
        )
    )
    return processed


processed = process_raw(data.groups["raw"])
pprint(processed)

MeasurementOutcomesDataGroup(
│   attrs={},
│   outcomes=Dataset(dtype='float32', shape=(3,), data=array([1., 1., 1.], dtype=float32), attrs={})
)

In [13]:
data.groups.update(processed=processed)
pprint(data)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(3, 2, 2),
│   │   │   │   data=array([[[0.24662715, 0.86970806],
│   │   [0.34476417, 0.90156907]],
│      
│      [[0.17104147, 0.60216415],
│   │   [0.50629896, 0.80135703]],
│      
│      [[0.9071514 , 0.8023807 ],
│   │   [0.68783087, 0.0879358 ]]], dtype=float32),
│   │   │   │   attrs={}
│   │   │   )
│   │   ),
│   │   'processed': MeasurementOutcomesDataGroup(
│   │   │   attrs={},
│   │   │   outcomes=Dataset(dtype='float32', shape=(3,), data=array([1., 1., 1.], dtype=float32), attrs={})
│   │   )
│   }
)

In [ ]:
def process_outcomes(
    measurements: MeasurementOutcomesDataGroup,
) -> ExpectationValueDataGroup:
    expval = ExpectationValueDataGroup(
        expectation_value=Dataset(
            shape=(), dtype="float32", data=measurements.outcomes.data.mean(),
            attrs={'date': '20', 'input': 10}
        )
    )
    return expval


expval = process_outcomes(processed)
data.groups.update(expval=process_outcomes(data.groups["processed"]))

pprint(expval)

ExpectationValueDataGroup(
│   attrs={},
│   expectation_value=Dataset(dtype='float32', shape=(), data=np.float32(1.0), attrs={})
)

In [15]:
pprint(data)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=Dataset(
│   │   │   │   dtype='float32',
│   │   │   │   shape=(3, 2, 2),
│   │   │   │   data=array([[[0.24662715, 0.86970806],
│   │   [0.34476417, 0.90156907]],
│      
│      [[0.17104147, 0.60216415],
│   │   [0.50629896, 0.80135703]],
│      
│      [[0.9071514 , 0.8023807 ],
│   │   [0.68783087, 0.0879358 ]]], dtype=float32),
│   │   │   │   attrs={}
│   │   │   )
│   │   ),
│   │   'processed': MeasurementOutcomesDataGroup(
│   │   │   attrs={},
│   │   │   outcomes=Dataset(dtype='float32', shape=(3,), data=array([1., 1., 1.], dtype=float32), attrs={})
│   │   ),
│   │   'expval': ExpectationValueDataGroup(
│   │   │   attrs={},
│   │   │   expectation_value=Dataset(dtype='float32', shape=(), data=np.float32(1.0), attrs={})
│   │   )
│   }
)

In [18]:
filepath = pathlib.Path("test.h5")
data.model_dump_hdf5(filepath)

In [19]:
data_reload = Datastore.model_validate_hdf5(filepath)
pprint(data_reload)

Datastore(
│   groups={
│   │   'raw': SinaraRawDataGroup(
│   │   │   attrs={'date': '2025-03-26', 'version': 0.1},
│   │   │   camera_images=array([[[0.24662715, 0.86970806],
│   │   [0.34476417, 0.90156907]],
│      
│      [[0.17104147, 0.60216415],
│   │   [0.50629896, 0.80135703]],
│      
│      [[0.9071514 , 0.8023807 ],
│   │   [0.68783087, 0.0879358 ]]], dtype=float32)
│   │   ),
│   │   'processed': MeasurementOutcomesDataGroup(attrs={}, outcomes=array([1., 1., 1.], dtype=float32)),
│   │   'expval': ExpectationValueDataGroup(attrs={}, expectation_value=array(1., dtype=float32))
│   }
)